# Decision Trees and Random Forests: An Overview

## Case Study: Bank Marketing Data

Given a dataset consisting of various features related to clients of a bank, we want to predict if a client will subscribe to a term deposit.

In [ ]:
# Standard libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

%matplotlib inline

#### Quiz: Read in the 'bank.csv' file using the appropriate command from the list (save it into a variable called data):
1. from_csv
2. to_csv
3. read_csv

In [ ]:
data = 

In [ ]:
# View the first 5 rows of the dataset
data.head()

In [ ]:
# Provides summary statistics on central tendency, dispersion, and shape of a dataset (ONLY NUMERICAL VARS).
data.describe()

In [ ]:
# Displays all the columns (features) of the dataset.
data.columns

## Understanding our features

Always make it a point to understand your features before you dive into your dataset! This will almost instantly give us a better of idea of what to look for in our visualizations, i.e., skewed data, inconsistencies, outliers, etc.

In our dataset, we have the following features:

**CATEGORICAL**
1. Job : What the person does, i.e., admin,technician, services, management, etc.
2. Marital : Martial Status, i.e., married, single, divorced
3. Education: Highest education secondary, tertiary, primary, unknown
4. Default : Whether or not the person has housing
5. Housing : Whether or not the person has housing
6. Loan : Whether or not the applicant has taken out a loan from the bank.
7. Deposit : yes, no (Dependent Variable)
8. Contact : Type of contact provided (unknown, cellular, telephone)
9. Month
10. poutcome: unknown, other, failure, success

**NUMERICAL**
1. Age: age of the person
2. balance: current bank balance
3. day: 
4. duration
5. campaign
6. pdays
7. previous

In [ ]:
# Check for any null/NAN values in the features
data[data.isnull().any(axis=1)].count()

## Let's visualize our features (remember we have to evaluate the past before predicting the future!)

#### Quiz: Extract the appropriate columns for the duration using the appropriate Pandas command:

In [ ]:
age = data.age
duration = 

In [ ]:
# Side-by-side plots of the age and the duration distributions.
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(12, 10))
sns.distplot(age, ax=ax[0])
sns.distplot(duration, ax=ax[1])

### Analyzing the plots above:

- Seems that some of the features like the age and the duration are highly skewed (look at the distribution plots).
- This does not matter when it comes to Decision Trees! Does anyone know why?

## Categorical Feature Modification and Encoding (Preparing our data for the model)

In [ ]:
def combine_jobs(data):
    '''
    Combine professions into a set of parent professions (white collar, pink collar, blue collar, etc.)
    '''
    # Extract each unique job value
    jobs = data.job.unique()
    jobs_count = data.job.value_counts()
    '''
    Iterate through and compute the ratio of people in a particular
    profession who have made a deposit.
    '''
#     for job in jobs:
#         print("{:15} : {:5}"
#               .format(job, len(data[(data.deposit == "yes") & (data.job == job)]) / jobs_count[job]))
        
    # Define the different 'parent' professions. Inspiration from blue collar
    white_collar = ['management', 'admin.', 'entrepreneur']
    pink_collar = ['services','housemaid']
    blue_collar = ['technician']
    others = ['retired', 'student', 'self-employed', 'unemployed', 'unknown']
    
    # Replace each of the above professions with their parent profession
    data['job'] = data['job'].replace(white_collar, 'white-collar')
    data['job'] = data['job'].replace(blue_collar, 'blue-collar')
    data['job'] = data['job'].replace(pink_collar, 'pink-collar')
    data['job'] = data['job'].replace(others, 'other')
    
#     print(data.job.value_counts())

In [ ]:
def drop_contact_vars(data):
    '''
    Drop columns related to any contact-related information that don't play a role in classification.
    '''
    # Drop the 'contact' column
    columns_to_drop = ['contact', 'month', 'day']
    for column in columns_to_drop:
        data.drop(column, axis=1, inplace=True)

In [ ]:
def encode_binary_variables(data):
    '''
    Encode binary categorical variables to their numeric equivalents.
    '''
    binary_variables = ['default', 'housing', 'loan', 'deposit']
    for binary_variable in binary_variables:
        data[binary_variable + '_cat'] = data[binary_variable].map({'yes': 1, 'no': 0})
        data.drop(binary_variable, axis=1, inplace=True)
        
#     for binary_variable in binary_variables:
#         print(data[binary_variable + '_cat'].value_counts())

In [ ]:
def remove_negative_pdays(data):
    '''
    pdays: number of days that passed by after the client was last contacted from a previous campaign
    -1 means client was not previously contacted
    '''
#     print("Customers that have not been contacted before:", len(data[data.pdays==-1]))
#     print("Maximum values on padys    :", data.pdays.max())
    
    # Map padys=-1 into a large value (10000 is used) to indicate that it is so far in the past that it has no effect
    data.loc[data.pdays == -1, 'pdays'] = 10000

def normalize_pdays(data):
    # Create a new column: recent_pdays (MOVE INTO PREV CELL)
    data['recent_pdays'] = np.where(data['pdays'], 1/data.pdays, 1/data.pdays)

    # Drop 'pdays'
    data.drop('pdays', axis=1, inplace = True)

In [ ]:
combine_jobs(data)
drop_contact_vars(data)
encode_binary_variables(data)
remove_negative_pdays(data)
normalize_pdays(data)

In [ ]:
sns.barplot(x='job', y = 'deposit_cat', data=data)

### One Hot Encoding

In [ ]:
# def one_hot_encode(data):
columns = ['job', 'marital', 'education', 'poutcome']
data = pd.get_dummies(data=data, columns = columns, prefix = ['job', 'marital', 'education', 'poutcome'])

In [ ]:
data.head()

In [ ]:
correlation_matrix = data.corr()

def plot_correlation_matrix(data):
    '''
    Plots the correlation matrix and visualize it using Seaborn's heatmap feature.
    '''
    # Plotting the correlation matrix using a heatmap
    plt.figure(figsize = (10,10))
    cmap = sns.diverging_palette(220, 10, as_cmap=True)
    sns.heatmap(correlation_matrix, xticklabels=correlation_matrix.columns.values, yticklabels=correlation_matrix.columns.values, cmap=cmap, vmax=.3, center=0, square=True, linewidths=.5, cbar_kws={"shrink": .82})
    plt.title('Heatmap of Correlation Matrix')
    
plot_correlation_matrix(data)

In [ ]:
DEPOSIT_COLUMN = 'deposit_cat'
def plot_deposit_correlations(data):
    '''
    Isolates the deposit columns of the correlation matrix and visualize it.
    '''
    deposit_correlation_column = pd.DataFrame(correlation_matrix[DEPOSIT_COLUMN].drop(DEPOSIT_COLUMN))
    deposit_correlation_column = deposit_correlation_column.sort_values(by=DEPOSIT_COLUMN, ascending=False)
    sns.heatmap(deposit_correlation_column)
    plt.title('Heatmap of Deposit Variable Correlations')
    
plot_deposit_correlations(data)

## Training the Model

In [ ]:
labels = data[DEPOSIT_COLUMN]
data.drop(DEPOSIT_COLUMN, axis=1, inplace=True)

#### Quiz: Fill in the parameters for the train_test_split with the following values:
1. **data** for the feature matrix
2. **labels** for the labels
3. **0.2** for the test_size
4. **50** for your random_state

In [ ]:
X_train, X_test, y_train, y_test = train_test_split()

In [ ]:
decision_tree_classifier = DecisionTreeClassifier(random_state=1)
decision_tree_classifier.fit(X_train, y_train)

In [ ]:
# Compute our training and test accuracy
training_score = decision_tree_classifier.score(X_train, y_train)
test_score = decision_tree_classifier.score(X_test, y_test)

print('Training score: ', training_score)
print('Test score: ', test_score)

## Tuning your Hyperparameters using Grid Search

Grid Search is an algorithm that takes in a list of different values for each hyperparameter, trains your model with every single combination of the hyperparameters, and outputs the best model accuracy.

In [ ]:
# Define a dictionary of parameters to be searched through
parameter_grid = {
    # Criteria for how we measure the quality of a split.
    'criterion': ['gini', 'entropy'],
    'max_depth': [2, 3, 4, 6],
    'max_leaf_nodes': [None, 4, 7, 10]
}

classifier = GridSearchCV(decision_tree_classifier, parameter_grid, cv=5)
classifier.fit(X_train, y_train)

In [ ]:
def grid_search_report(classifier):
    '''
    Generate a report on the performance of the classifier on each combination of parameters.
    '''
    means = classifier.cv_results_['mean_test_score']
    stds = classifier.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, classifier.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))
        
    print('Best performing hyperparameters: ', classifier.best_params_)
    
grid_search_report(classifier)

In [ ]:
def test_accuracy_report(classifier):
    '''
    Generate a report on how the best classifier performs on the test set.
    '''
    y_pred = classifier.predict(X_test)
    print('Accuracy score of best classifier: ', accuracy_score(y_test, y_pred))
    
test_accuracy_report(classifier)

## Doing even better with Ensemble Learning: Random Forests!

Ensemble learning simply combines the output of a set of weak learners to produce a stronger learner. In our case, we shall combine 100 Decision Trees and produce a model out of it.

In [ ]:
rf_classifier = RandomForestClassifier(n_estimators=100, max_depth=6, criterion='entropy', random_state=50)
rf_classifier.fit(X_train, y_train)

In [ ]:
test_accuracy_report(rf_classifier)

## Resources and Further Readings

[Ensemble Learning Explained](https://blog.statsbot.co/ensemble-learning-d1dcd548e936)  
[Decision Trees Explained Simply](https://medium.com/@chiragsehra42/decision-trees-explained-easily-28f23241248)  
[Understanding Random Forests](https://towardsdatascience.com/understanding-random-forest-58381e0602d2)